In [7]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os

In [15]:
file = r'./data_results\Customer_tol01_largefx_12V.xlsx'
def recup_variable(file):
    a = pd.read_excel(file)
    data_variable = {}
    for var in a[(a["Type"]=='variable')&(a["Count"]>1)]['Name']:
        data_variable[var] = pd.read_excel(file,sheet_name=var,header=2)
    return data_variable
def recup_parameter(file):
    a = pd.read_excel(file)
    data_variable = {}
    for var in a[(a["Type"]=='parameter')&(a["Count"]>1)]['Name']:
        data_variable[var] = pd.read_excel(file,sheet_name=var,header=2)
    return data_variable
def recup_set(file):
    a = pd.read_excel(file)
    data_variable = {}
    for var in a[(a["Type"]=='set')&(a["Count"]>1)]['Name']:
        data_variable[var] = pd.read_excel(file,sheet_name=var,header=2)
    return data_variable
data_variable = recup_variable(file)
data_parameter = recup_parameter(file)
data_set = recup_set(file)

In [22]:
data_parameter['psi']

,c,degree,Value
0,small,0,152.324500
1,small,2,-0.001039
2,large,0,178.351600
3,large,2,-0.000370


In [29]:
data_variable['Qpompe']['Value'].max()

271.7795010837408

In [30]:
data_parameter['psi']
Qmax = int(data_variable['Qpompe']['Value'].max()*1.1)
X = np.linspace(0,Qmax,Qmax)
Y = data_parameter['psi'][data_parameter['psi']['degree']==2]['Value'].iloc[0]*X**2 + data_parameter['psi'][data_parameter['psi']['degree']==0]['Value'].iloc[0]
color_map = {
    'small' : 'blue',
    'large' : 'red'
}
fig = go.Figure()
for c in data_variable['Qpompe']['c'].unique():
    fig.add_trace(
        go.Scatter(
            x = X,
            y = data_parameter['psi'][(data_parameter['psi']['degree']==2)&(data_parameter['psi']['c']==c)]['Value'].iloc[0]*X**2 + data_parameter['psi'][(data_parameter['psi']['degree']==0)&(data_parameter['psi']['c']==c)]['Value'].iloc[0],
            marker_color = color_map[c],
            name = f'Courbe de gain de charge théorique {c}'
        )
    )
for c in data_variable['Qpompe']['c'].unique():
    for d in data_variable['Qpompe']['d'].unique():
        fig.add_trace(
            go.Scatter(
                x               =   data_variable['Qpompe'][(data_variable['Qpompe']['c']==c)&(data_variable['Qpompe']['d']==d)]['Value'],
                y               =   data_variable['Charge'][data_variable['Charge']['n']=='s']['Value'],
                marker_color    =   color_map[c],
                name            =   f"Débit {c},{d}",
                # hoverinfo       =   data_variable['Qpompe'][(data_variable['Qpompe']['c']==c)&(data_variable['Qpompe']['d']==d)]['t'],
                mode='markers'
            )
        )
fig.update_layout(
    height= 800,
    hovermode='x'
)
# fig.update_xaxes(range=[0,data_variable['Qpompe']['Value'].max()*1.1])
# fig.update_yaxes(range=[data_variable['Charge'][data_variable['Charge']['n']=='s']['Value'].min()*0.9,data_variable['Charge'][data_variable['Charge']['n']=='s']['Value'].max()*1.1])
fig.show()

In [70]:
data_variable['Charge'][data_variable['Charge']['n']=='s']['Value']

0     152.048130
1     152.047536
2     152.047333
3     152.047255
4     152.047160
5     152.052122
6     152.063275
7     152.079546
8     145.840219
9     147.132310
10    151.143323
11    151.125791
12    151.106413
13    151.091649
14    151.077808
15    151.065812
16    151.053816
17    151.047818
18    151.036902
19    151.029520
20    151.021884
21    151.035698
22    151.031841
23    151.025495
Name: Value, dtype: float64

In [72]:
data_parameter['psi'].query(f"c=='{c}' & degree==0")['Value'].iloc[0] - data_parameter['psi'].query(f"c=='{c}' & degree==2")['Value'].iloc[0]*data_variable['Qpompe'].query(f"c=='{c}' & d=='{d}'")['Value']**2

120    178.351970
121    178.679993
122    204.655853
123    179.462034
124    179.053045
125    181.216839
126    204.639925
127    184.833057
128    178.351970
129    178.387071
130    178.463949
131    185.022948
132    178.521298
133    178.351970
134    178.353000
135    179.685856
136    178.748250
137    180.027517
138    178.351970
139    178.448285
140    179.825298
141    178.757119
142    179.153170
143    179.997128
Name: Value, dtype: float64

In [73]:
pompe = []
débit_pompe = {}
for c in data_variable['Qpompe']['c'].unique():
    for d in data_variable['Qpompe'][data_variable['Qpompe']['c']==c]['d'].unique():
        print(data_parameter['psi'].query(f"c=='{c}' & degree==0")['Value'].iloc[0])
        débit_pompe[(c,d)] = (data_variable['Charge'][data_variable['Charge']['n']=='s']['Value'].to_numpy()/(data_parameter['psi'].query(f"c=='{c}' & degree==0")['Value'].iloc[0] - data_parameter['psi'].query(f"c=='{c}' & degree==2")['Value'].iloc[0]*data_variable['Qpompe'].query(f"c=='{c}' & d=='{d}'")['Value'].to_numpy()**2) - 1)

liste

152.3245
152.3245
178.3516
178.3516
178.3516
178.3516


[('small', 'p1'),
 ('small', 'p2'),
 ('large', 'p1'),
 ('large', 'p2'),
 ('large', 'p3'),
 ('large', 'p4')]

In [58]:
color_map   =   {
        'p1'    :   'red',
        'p2'    :   'blue',
        'p3'    :   'green',
        'p4'    :   'magenta',
    }

In [74]:
débit_pompe[('large','p1')]

array([-0.25705173, -0.25705679, -0.17648578, -0.25705918, -0.25705999,
       -0.25701773, -0.15699321, -0.25678414, -0.18228983, -0.251365  ,
       -0.26472362, -0.26487161, -0.15276286, -0.20653741, -0.20778481,
       -0.2653777 , -0.26547888, -0.18978183, -0.21515755, -0.26568378,
       -0.16162525, -0.26563168, -0.26566421, -0.26212256])

In [77]:
fig = go.Figure()
for pompe in débit_pompe:
    fig.add_trace(go.Box(y=débit_pompe[pompe], name=f"{pompe}"))
fig.update_layout(title = "Ecart relatif sur la charge des pompes induit par la relaxation convexe")
fig.show()

In [142]:
file = './data_results/4WT_all_fx.xlsx'
a = pd.read_excel(file,sheet_name='comments')
a.iloc[0,0]

"Optimisation du problème 4WT. En imposant que toutes les pompes soient allumées la nuit et que la pompe 1 soit tout le temps allumée. Le temps de calcul est de quelques secondes. La solution est réalisable. L'écart relatif de avec la solution optimale est de moins de 10E-4."

In [65]:
data_parameter['demand'][data_parameter['demand']['r']=='r2']

,r,t,Value
24,r2,t1,44.83
25,r2,t2,18.00
26,r2,t8,45.00
27,r2,t9,51.67
28,r2,t13,15.17
29,r2,t14,103.83
30,r2,t15,34.83
31,r2,t16,43.83
32,r2,t17,54.83
33,r2,t18,51.17


,t,r,Value
0,t1,r2,44.83
1,t2,r2,18.00
2,t3,NaN,NaN
3,t4,NaN,NaN
4,t5,NaN,NaN
5,t6,NaN,NaN
6,t7,NaN,NaN
7,t8,r2,45.00
8,t9,r2,51.67
9,t10,NaN,NaN


In [132]:
n = 'r2'
def path_to_n(n,data_set):
    path = [n]
    while n !='s':
        n = data_set['l'][data_set['l']['n.1']==n]['n'].iloc[0]
        path.append(n)
    return path

fig = go.Figure()
path = path_to_n(n,data_set)
fig.add_trace(
    go.Scatter(
        x = path,
        y = pd.concat([pd.DataFrame([['s',0,0]],columns=data_parameter['height'].columns),data_parameter['height'][data_parameter['height']['n'].isin(path)]])['Value'],
        name = 'hauteur'
    )
)
for t in data_set['t']['dim1']:
    if t == 't1':
        fig.add_trace(
        go.Scatter(
            x = path,
            y = data_variable['Charge'][(data_variable['Charge']['n'].isin(path))&(data_variable['Charge']['t']=='t1')]['Value'],
            name = t,
            # visible='legendonly'
        )
        )
    else:
        fig.add_trace(
        go.Scatter(
            x = path,
            y = data_variable['Charge'][(data_variable['Charge']['n'].isin(path))&(data_variable['Charge']['t']==t)]['Value'],
            name = t,
            visible='legendonly'
        )
        )
fig.update_layout(
    hovermode='x',
    # width=400
)
fig.show()

In [87]:
liste_reservoir = data_set['r']['n']


for i,reservoir in enumerate(liste_reservoir):
    fig = go.Figure()
    demand = pd.merge(data_set['t'][['dim1']].rename(columns={'dim1':'t'}),data_parameter['demand'][data_parameter['demand']['r']==reservoir],on=['t'],how='left')
    demand['Value'] = demand['Value'].fillna(0)
    fig.add_trace(
        go.Scatter(
            x = demand['t'],
            y = -demand['Value'],
            stackgroup='one',
            name = 'Demande'
        )
    )

    fig.add_trace(
        go.Scatter(
            x = data_set['t']['dim1'],
            y = sum(data_variable['Qpipe'][(data_variable['Qpipe']['n.1']==reservoir)&(data_variable['Qpipe']['n']==n)]['Value'] for n in data_set['l'][data_set['l']['n.1']==reservoir]['n']),
            stackgroup='two',
            name = 'Approvisionnement'
        )
    )
    fig.add_trace(
        go.Scatter(
            x = data_set['t']['dim1'],
            y = data_variable['v'][data_variable['v']['n']==reservoir]['Value']-data_variable['v'][data_variable['v']['n']==reservoir]['Lowerbound'],
            name = 'Volume - Volume min'
        )
    )
    fig.update_layout(
        hovermode='x',
        legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
                ),
        title = reservoir
    )
    fig.show()

In [13]:
Z = 2
fig = go.Figure()
for r in data_variable['v']['n'].unique():
    fig.add_trace(
        go.Bar(
            x   =  data_variable['v'][data_variable['v']['n']==r]['t'],
            y   =  data_variable['v'][data_variable['v']['n']==r]['Value'],
            name=   r
        )
    )
for c in data_variable['Ppompe']['c'].unique():
    for d in data_variable['Ppompe']['d'].unique():
        fig.add_trace(
            go.Scatter(
                x   =   data_variable['Ppompe'][(data_variable['Ppompe']['c']==c)&(data_variable['Ppompe']['d']==d)]['t'],
                y   =   data_variable['Ppompe'][(data_variable['Ppompe']['c']==c)&(data_variable['Ppompe']['d']==d)]['Value'],
                yaxis="y2",
                name=   f"({c},{d})"
            )
        )
fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
            ),
        hovermode='x',
        yaxis=dict(
            title="m3" ,
            # range=[0,df["Consommation (MW)"].max()*1.1]
            ),
        yaxis2=dict(title="kW",
        # range=[df["Coût MWh"].min()*0.95,df["Coût MWh"].max()*1.1],
        anchor="free",
        overlaying="y",
        side="right",
        position=1
        ),
        title = f"Etat du réseau de distribution d'eau - Coût : {Z} €"
    )
fig.show()
# fig.write_html("result.html")

In [56]:
[file for file in os.listdir('./data_results') if file.endswith('.xlsx')][0]

'4WT_fx_night.xlsx'

In [64]:
data_variable['Charge'][data_variable['Charge']['n']=='s']['Value']

0     57.635219
1     54.299841
2     54.558968
3     54.949131
4     55.367206
5     55.807872
6     56.265675
7     56.385046
8     53.760826
9     60.929469
10    53.366659
11    58.832858
12    61.051633
13    60.923149
14    55.287573
15    60.908369
16    60.909363
17    63.073191
18    63.073191
19    63.073191
20    63.073191
21    58.391783
22    63.073191
23    59.646449
Name: Value, dtype: float64

In [23]:
color_map   =   {
    'p1'    :   'red',
    'p2'    :   'blue',
    'p3'    :   'green'
}
color_map_light   =   {
    'p1'    :   'lightcoral',
    'p2'    :   'lightblue',
    'p3'    :   'lightgreen'
}
fig = go.Figure()
for c in data_variable['Son']['c'].unique():
    for d in data_variable['Son']['d'].unique():
        fig.add_trace(
            go.Bar(
                x               =   data_variable['Son'][(data_variable['Son']['c']==c)&(data_variable['Son']['d']==d)]['t'],
                y               =   10*data_variable['Son'][(data_variable['Son']['c']==c)&(data_variable['Son']['d']==d)]['Value'],
                marker_color    =   color_map_light[d],
                name            =   f"Statut {c},{d}"
            )
        )
fig.update_layout(
    barmode="stack"
)
fig.add_trace(
    go.Scatter(
        x               =   data_variable['Gpompe']['t'],
        y               =   data_variable['Gpompe']['Value'],
        marker_color    =   'black',
        line_dash       =   'dash',
        name            =   f"Gain Charge {c},{d}"
    )
)
for c in data_variable['Qpompe']['c'].unique():
    for d in data_variable['Qpompe']['d'].unique():
        fig.add_trace(
            go.Scatter(
                x               =   data_variable['Qpompe'][(data_variable['Qpompe']['c']==c)&(data_variable['Qpompe']['d']==d)]['t'],
                y               =   data_variable['Qpompe'][(data_variable['Qpompe']['c']==c)&(data_variable['Qpompe']['d']==d)]['Value'],
                marker_color    =   color_map[d],
                name            =   f"Débit {c},{d}"
            )
        )
fig.update_layout(
    hovermode='x',
    title = f"Etat des pompes du réseau de distribution d'eau - Coût : {Z} € "
    )
fig.show()

In [ ]:
fig = go.Figure()
for n in data_variable['Charge']['n'].unique():
    fig.add_trace(
        go.Scatter(
            x = data_variable['Charge'][data_variable['Charge']['n']==n]['t'],
            y = data_variable['Charge'][data_variable['Charge']['n']==n]['Value'],
            name = n
        )
    )
fig.update_layout(
    legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
            ),
    hovermode='x',
    title = f"Charge dans le réseau de distribution d'eau - Coût : {Z} € ",
    yaxis_title = "Charge (m)"
)
fig.show()